In [1]:
!nvidia-smi

Sat Jun 26 16:51:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!python -c "import torch; print(torch.version.cuda)"

10.2


In [3]:
!python -c "import torch; print(torch.__version__)"

1.9.0+cu102


###**Imports**

In [4]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

!pip install -q path
!pip install torchmetrics

%matplotlib inline
import os
import numpy as np
from path import Path
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torchmetrics import Accuracy, IoU
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import ShapeNet
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.utils import to_dense_batch

import datetime
import itertools
import tensorflow
import tensorboard
from time import time
from tensorboard import notebook
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
%reload_ext tensorboard

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


###**Hyperparameters**

In [5]:
hparams = {
    'k': 3,
    'num_classes': 16,
    'lr': 1e-3,
    'wd': 1e-3,
    'bs': 16,
    'num_workers': 2,
    'shuffle_train': False,
    'shuffle_valid': False,
    'epochs': 20,
    'schedule': False
}

###**PointNet**

In [12]:
class TNet(nn.Module):
  def __init__(self, k=3):
    super().__init__()
    self.k = k
    self.sharedMLP = nn.Sequential(
        nn.Conv1d(in_channels=k, out_channels=64, kernel_size=1, bias=True),
        nn.BatchNorm1d(64), 
        nn.ReLU(),
        nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1, bias=True),
        nn.BatchNorm1d(128), 
        nn.ReLU(),
        nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1, bias=True),
        nn.BatchNorm1d(1024), 
        nn.ReLU()
    )
    self.FC = nn.Sequential(
        nn.Linear(in_features=1024, out_features=512, bias=True),
        nn.BatchNorm1d(512), 
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=256, bias=True),
        nn.BatchNorm1d(256), 
        nn.ReLU(),
        # nn.Dropout(p=0.7), only apply when overfitting
        nn.Linear(in_features=256, out_features=k*k), 
    )

  def forward(self, cloud_points):
    bs = cloud_points.size(0)
    x = self.sharedMLP(cloud_points)
    # size: [batch size, 1024, # of points]
    x = nn.MaxPool1d(x.size(-1))(x) # pool with kernel = # of points/batch
    # size: [batch size, 1024, 1]
    x = nn.Flatten(start_dim=1)(x) # flatten to get horizontal vector
    # size: [batch size, 1024]
    x = self.FC(x)
    # diagonal matrices initialized, as many as batch size
    init_matrix = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
    if torch.cuda.is_available() and x.is_cuda:
      init_matrix = init_matrix.cuda() # gets updated according to f.c. output
    matrix = x.view(-1, self.k, self.k) + init_matrix
    return matrix

class Transform(nn.Module):
   def __init__(self, k=3):
        super().__init__()
        self.k = k
        # Input transform + First Shared MLP:
        self.input_transform = TNet(k)
        self.fc1 = nn.Sequential(
          nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1, bias=True),
          nn.ReLU(),
          nn.BatchNorm1d(64)
        )
        # Feature transform + Second Shared MLP: 
        self.feature_transform = TNet(k=64)
        self.fc2 = nn.Sequential(
          nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1, bias=True),
          nn.BatchNorm1d(128), 
          nn.ReLU(),
          nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1, bias=True),
          nn.BatchNorm1d(1024)
        )

   def forward(self, x):
        bs = x.size(0)
        # ------------------------------------------------------------
        matrix3x3 = self.input_transform(x)
        x = torch.bmm(torch.transpose(x,1,2),matrix3x3).transpose(1,2)
        x = self.fc1(x)
        # ------------------------------------------------------------
        matrix64x64 = self.feature_transform(x)
        y = torch.bmm(torch.transpose(x,1,2), matrix64x64).transpose(1,2) 
        x = self.fc2(y)
        # Maxpool 
        y = nn.MaxPool1d(x.size(-1))(x)
        """
        x = nn.Flatten(1)(x).repeat(n_points,1,1).transpose(0,2).transpose(0,1)
        y = torch.cat((x, y.repeat(1,1,n_points)), 1)
        """
        
        global_features = x.view(bs,-1)
        return y

class PointNetModel(nn.Module):
    def __init__(self, k=3, num_classes=16):
        super().__init__()
        self.k = k
        # Transform + Last MLP
        
        self.transform = Transform(self.k)
        '''
        self.FC1 = nn.Linear(in_features=1024, out_features=512)
        self.bn1 = nn.BatchNorm1d(512)
        self.FC2 = nn.Linear(in_features=512, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.FC3 = nn.Linear(in_features=256, out_features=num_classes)
        self.dropout = nn.Dropout(p=0.3)
        '''
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.Dropout(p=0.3),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes),
        )
        """
        self.fc2 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=num_classes, kernel_size=1)
        )
        """
    def forward(self, x):
      '''
      x = self.transform(x)
      x = F.relu(self.bn1(self.FC1(x))) 
      x = F.relu(self.bn2(self.dropout(self.FC2(x)))) 
      output = self.FC3(x)
      '''
      x = self.transform(x)
      output = self.fc1(x)
      
      return output, F.softmax(output,dim=1)

###**ShapeNet**

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sn_train = ShapeNet(root='/content/drive/MyDrive/Dataset_ShapeNet/',split="train",include_normals=False)
sn_valid = ShapeNet(root='/content/drive/MyDrive/Dataset_ShapeNet/',split="val",include_normals=False)

Mounted at /content/drive


In [9]:
def get_loader(data, 
               bs=1, 
               num_workers=2, 
               shuffle=False):
  # Function to load ShapeNet split into a DataLoader object:
  data_loader = DataLoader(data, 
                          batch_size=bs,
                          shuffle=shuffle,
                          num_workers=num_workers)
  return data_loader

device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_loader = get_loader(sn_train)
model = PointNetModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)

for i, data in enumerate(train_loader,1):
  datos = data
  targets = datos.category.to(device)
  points = datos.pos.unsqueeze(2).to(device)
  #targets = datos.category.to(device)
  preds, probs = model(points)
  print(preds.shape)
  print(targets.shape)
  #print(preds.squeeze(-1))
  print(criterion(preds.squeeze(-1), targets))

RuntimeError: ignored

In [ ]:
for i, data in enumerate(train_loader):
  targets = []
  counts = torch.unique(data.batch, return_counts=True)[1]
  for i, subdata in enumerate(data.category):
    targets.append(subdata.repeat(counts[i]))
  targets = torch.cat(targets)
  print(targets)

###**Train Loop**

In [10]:
def get_loader(data, 
               bs=32, 
               num_workers=2, 
               shuffle=False):
  # Function to load ShapeNet split into a DataLoader object:
  data_loader = DataLoader(data, 
                          batch_size=bs,
                          shuffle=shuffle,
                          num_workers=num_workers)
  return data_loader

def train_epoch(model, 
                train_loader, 
                optimizer, 
                device, 
                criterion, 
                accuracy):
  # List for epoch loss:
  epoch_train_loss = []
  # Model in train mode:
  model.train()
  # Metrics stored information reset:
  accuracy.reset()
  #iou.reset()
  # Batch loop for training:
  for i, data in enumerate(train_loader,1):
      # Data retrieval from each bath:   
      points = to_dense_batch(data.pos,batch=data.batch)[0].to(device).float().transpose(1,2)
      targets = data.category.to(device)
      # Forward pass:
      preds, probs = model(points)
      # Loss calculation + Backpropagation pass
      optimizer.zero_grad()
      loss = criterion(preds.squeeze(-1), targets).to(device)
      epoch_train_loss.append(loss.item()) # save loss to later represent
      loss.backward()
      optimizer.step()
      # Batch metrics calculation:
      accuracy.update(probs.squeeze(),targets)
      #iou.update(probs.squeeze(), targets)
  # Mean epoch metrics calculation:
  mean_loss = np.mean(epoch_train_loss)
  mean_acc = accuracy.compute().item()
  #mean_iou = iou.compute().item()
  # Print of all metrics:
  print('Train loss: ', mean_loss, "| Acc.: ", mean_acc)
  return mean_loss, mean_acc

def valid_epoch(model, 
                valid_loader, 
                scheduler,
                schedule, 
                device,
                criterion, 
                accuracy):
  # List for epoch loss:
  epoch_valid_loss = []
  # Model in validation (evaluation) mode:
  model.eval()
  # Metrics stored information reset:
  accuracy.reset()
  #iou.reset()
  # Batch loop for validation:
  with torch.no_grad():
    for i,data in enumerate(valid_loader,1):
        # Data retrieval from each bath:   
        #points = data.pos.unsqueeze(2).to(device)
        points = to_dense_batch(data.pos,batch=data.batch)[0].to(device).float().transpose(1,2)
        #targets = data.category.to(device)
        # Forward pass:
        preds, probs = model(points)
        # Loss calculation + Backpropagation pass
        loss = criterion(preds.squeeze(-1), targets.repeat(points.shape[0])).to(device)
        epoch_valid_loss.append(loss.item())
        # Batch metrics calculation:
        accuracy.update(probs.squeeze(),targets)
        #iou.update(probs.squeeze(),targets)
  # Mean epoch metrics calculation:
  mean_loss = np.mean(epoch_valid_loss)
  mean_acc = accuracy.compute().item()
  #mean_iou = iou.compute().item()
  # Learnign rate adjustment with scheduler:
  if schedule == True: scheduler.step(mean_loss)
  # Print of all metrics:
  print('Valid loss: ', mean_loss, "| Acc.: ", mean_acc)
  return mean_loss, mean_acc

def fit(train_data, 
        valid_data,
        k=3,
        num_classes=16, 
        bs=128, 
        num_workers=2, 
        epochs=20, 
        lr=0.001,
        schedule=False, 
        shuffle_train=True, 
        shuffle_valid=False, 
        model_filename=None):
  # Data Loaders for train and validation:
  train_loader = get_loader(data=train_data, 
                            bs=bs, 
                            num_workers=num_workers, 
                            shuffle=shuffle_train)
  valid_loader = get_loader(data=valid_data, 
                            bs=bs, 
                            num_workers=num_workers, 
                            shuffle=shuffle_valid)
  
  # Features:
  device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  model = PointNetModel(k=k, num_classes=num_classes).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  scheduler = ReduceLROnPlateau(optimizer, patience=5)
  criterion = nn.CrossEntropyLoss().to(device)
  # Metrics:
  accuracy = Accuracy(average='micro', compute_on_step=False).to(device)
  #iou = IoU(num_classes=num_classes, absent_score=1, compute_on_step=False).to(device)
  # Tensorboard Block:
    # Avoid tensorboard crashing when adding embeddings:
  tensorflow.io.gfile = tensorboard.compat.tensorflow_stub.io.gfile 
  tensorboard_name = "tensorboad" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.csv'
  logdir = os.path.join(Path('/content/drive/MyDrive/Dataset_ShapeNet/'), tensorboard_name)
  writer = SummaryWriter(log_dir=logdir) # writer for each epoch
  #writer_best = SummaryWriter(log_dir=logdir) # writer for best IoU value
  
  # Lists for epoch metrics:
  train_loss = []
  valid_loss = []
  train_macc = []
  valid_macc = []
  #train_miou = []  
  #valid_miou = []
  
  # Initialize value for Acc (which is used to save the model of best performance)
  min_macc = 0
  # Start training:
  print('Start training...')
  for epoch in range(epochs):
    print("Epoch: ", epoch)
    # Training epoch:
    epoch_train_loss, epoch_train_macc = train_epoch(model, 
                                                    train_loader, 
                                                    optimizer, 
                                                    device, 
                                                    criterion, 
                                                    accuracy)
    # Validation epoch:
    epoch_valid_loss, epoch_valid_macc= valid_epoch(model,
                                                    valid_loader, 
                                                    scheduler, schedule, 
                                                    device, 
                                                    criterion, 
                                                    accuracy)
    # Add epoch metrics to lists:
    train_loss.append(epoch_train_loss)
    valid_loss.append(epoch_valid_loss)
    train_macc.append(epoch_train_macc)
    valid_macc.append(epoch_valid_macc)
    #train_miou.append(epoch_train_miou)
    #valid_miou.append(epoch_valid_miou)
    # Function to write metrics in selected tensorboard:
    #def write_metric_tb(tb, label, metric, epoch):
    #  tb.add_scalar(label, np.array(metric), epoch)
      # Write training and validation loss in tensorboard
    #write_metric_tb(writer, 'Train loss', epoch_train_loss, epoch)
    #write_metric_tb(writer, 'Valid loss', epoch_valid_loss, epoch)
      # Write mean IoU and accuracy in tensorboard
    #write_metric_tb(writer, 'Train Accuracy', epoch_train_macc, epoch)
    #write_metric_tb(writer, 'Valid Accuracy', epoch_valid_macc, epoch)
    #write_metric_tb(writer, 'Train IoU', epoch_train_miou, epoch)
    #write_metric_tb(writer, 'Valid IoU', epoch_valid_miou, epoch)
      # Save model and write in tensorboard: best if condition for IoU is fullfilled:
    if epoch_valid_macc > min_macc:
      min_maccu = epoch_valid_macc
      '''
      # Write training and validation loss in tensorboard: best
      write_best_metric_tb(writer_best, 'Train loss', epoch_train_loss, epoch)
      write_best_metric_tb(writer_best, 'Valid loss', epoch_valid_loss, epoch)
      # Write mean IoU and accuracy in tensorboard: best
      write_best_metric_tb(writer_best, 'Train Accuracy', epoch_train_macc, epoch)
      write_best_metric_tb(writer_best, 'Valid Accuracy', epoch_valid_macc, epoch)
      write_best_metric_tb(writer_best, 'Train IoU', epoch_train_miou, epoch)
      write_best_metric_tb(writer_best, 'Valid IoU', epoch_valid_miou, epoch)
      '''
      # Save model:
      #filename = model_filename if model_filename is not None else 'model_'+datetime.datetime.now().strftime("%Y%m%d")+'.pth'
      #save_path = os.path.join(Path('/content/drive/MyDrive/Dataset_ShapeNet/'), filename)
      #torch.save(model.state_dict(), save_path)
      #print('Model with %d loss overwritten. Model with %d loss saved to %s' %(min_macc, epoch_valid_loss, save_path))
    
    print('---------------------------------------------------------------------------------------------------------')
  
  # Plot for Losses:  
  fig = plt.figure()
  epochs = len(train_loss)
  plt.plot(range(epochs), train_loss, 'bo', label='Train loss', ms=3)
  plt.plot(range(epochs), valid_loss, 'ro', label='Valid loss', ms=3)
  plt.title('Training and Validation Loss')
  plt.legend()
  # Plots for Accuracy and IoU: 
  def print_metrics(metric1, metric2, label1, label2):
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(range(epochs), metric1, 'go', ms=3)
    ax2.plot(range(epochs), metric2, 'yo', ms=3)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel(str(label1), color='g')
    ax2.set_ylabel(str(label2), color='y')
  print_metrics(valid_macc, train_macc, 'Valid Point Accuracy', 'Train Point Accuracy')

  %tensorboard --logdir logs

###**Experiment**

In [13]:
fit(sn_train, sn_valid, hparams['k'], hparams['num_classes'], hparams['bs'], hparams['num_workers'], hparams['epochs'], hparams['lr'], hparams['schedule'], hparams['shuffle_train'], hparams['shuffle_valid'])

Start training...
Epoch:  0


RuntimeError: ignored

In [ ]:
1+1